In [1]:
import pickle as pkl
import os
#
import pandas as pd
import numpy as np
import random
rng = 3
random.seed(rng)

In [2]:
fname = "./ncmf_sim_data/dict_name_dataset_1.pkl"

In [3]:
data_dict_all = pkl.load(open(fname,"rb"))

In [4]:
data_dict_all.keys()

dict_keys(['dt1', 'ds1', 'ds2', 'ds3', 'dn1', 'dn2', 'dn3'])

#### User inputs (Values to be modified as per the dataset)

In [5]:
for dataset_name in data_dict_all.keys():
    #dataset_name = "dt1"
    data_dict = data_dict_all[dataset_name]
    sample_id = 1
    data_folder = "./ncmf_sim_data/"+dataset_name+"/"
    all_matrices = ["X0","X1","X2","X3","X4"]
    entity_mapping = {"X0":["E0","E1"],
                      "X1":["E0","E2"],
                      "X2":["E0","E3"],
                      "X3":["E4","E1"],
                      "X4":["E5","E2"]}
    entity_name_mapping = {'E0': "ent0", 
                           "E1": "ent1", 
                           "E2": "ent2",
                           "E3": "ent3",
                           "E4": "ent4",
                           "E5": "ent5"}
    target_matrix_index = [0,2]
    rng = 3
    #
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)
    #
    # Output files
    sampled_node_file = data_folder + f'sampled{sample_id}_node.dat'
    sampled_link_file = data_folder + f'sampled{sample_id}_link.dat'
    if target_matrix_index == -1:
        sampled_link_test_file = data_folder + f'sampled{sample_id}_link.dat.test'
    else:
        sampled_link_test_files = []
        for m in target_matrix_index:
            sampled_link_test_files.append(data_folder + f'sampled{sample_id}_link.dat.test.{m}')
    sampled_label_file = data_folder + f'sampled{sample_id}_label.dat'
    sampled_label_test_file = data_folder + f'sampled{sample_id}_label.dat.test'
    sampled_meta_file = data_folder + f'sampled{sample_id}_meta.dat'
    sampled_info_file = data_folder + f'sampled{sample_id}_info.dat'
    #
    sampled_dict_id_idx_file = data_folder + f"sampled{sample_id}_dict_id_idx.pkl" 
    #
    for i in range(len(all_matrices)):
        #exec(f"X{i} = pd.read_csv(data_folder + all_matrices[i], header = None).to_numpy()")
        exec(f"X{i} = data_dict[str({i})]['X']")
        # This block assigns each of the matrices to variables X0, X1, X2,...
    #
    for i in range(len(all_matrices)):
        print(f"Shape of X{i}", end = " ")
        exec(f"print(X{i}.shape)")
    #
    # Get entity IDs - uses matrix_entity_similarity to get total number of unique entries
    total_entities = 0
    matrices_seen = []
    for k,v in entity_mapping.items():
        if v[0] not in matrices_seen:
            matrices_seen.append(v[0])
            total_entities += eval(f"{k}.shape[0]")
            exec(f"{v[0]}_size = {k}.shape[0]")
        if v[1] not in matrices_seen:
            matrices_seen.append(v[1])
            total_entities += eval(f"{k}.shape[1]")
            exec(f"{v[1]}_size = {k}.shape[1]")
    print("total_entities: ",total_entities)
    #
    entity_df = pd.DataFrame(columns = ["entity_name","idx"])
    for E_id in matrices_seen:
        exec(f"cur_entity_size = {E_id}_size")
        exec(f"print('{E_id}_size: ',{E_id}_size)")
        for i in range(cur_entity_size):
            entity_df = entity_df.append({"entity_name": f"{E_id}_{i}","idx":f"{i}" }, ignore_index = True)
    #
    dict_id_idx = entity_df["idx"].astype(int).to_dict()
    full_df = pd.DataFrame(columns = ["left", "right", "value", "link_type"])
    #
    for k,v in entity_mapping.items(): # iterate over all matrices
        print(f"Considering {k}")
        link_type = int(k[1:])
        total_left_ents_so_far = 0
        for m in matrices_seen:
            if v[0] == m:
                break
            else:
                total_left_ents_so_far += eval(f"{m}_size")
        print(f"{v[0]} --> {total_left_ents_so_far}")
        total_right_ents_so_far = 0
        for m in matrices_seen:
            if v[1] == m:
                break
            else:
                total_right_ents_so_far += eval(f"{m}_size")
        print(f"{v[1]} --> {total_right_ents_so_far}")
    #     for i in range(eval(f"{v[0]}_size")):
    #         for j in range(eval(f"{v[1]}_size")):
    #             left_id = i + total_left_ents_so_far
    #             right_id = j + total_right_ents_so_far
    #             value = eval(f"{k}[i][j]")
    #             print(f"left = {left_id}, right = {right_id}, value = {value}")
    #             full_df = full_df.append({"left": left_id, "right": right_id, "value": value, "link_type": link_type}, ignore_index = True)
        temp_df = pd.DataFrame(columns = ["left", "right", "value", "link_type"])
        temp_df["left"] = sorted(list(range(total_left_ents_so_far, total_left_ents_so_far + eval(f"{v[0]}_size"))) * eval(f"{v[1]}_size"))
        temp_df["right"] = list(range(total_right_ents_so_far, total_right_ents_so_far + eval(f"{v[1]}_size"))) * eval(f"{v[0]}_size")
        temp_df["value"] = eval(f"{k}").flatten()
        temp_df["link_type"] = [link_type] * len(temp_df["left"])
        full_df = pd.concat([full_df, temp_df], axis = 0, ignore_index = True)
    #
    np.random.seed(rng)
    if target_matrix_index != -1:
        sample = {}
        for m in range(len(target_matrix_index)):
            target_matrix_R_ent_count = eval(f"X{target_matrix_index[m]}.shape[0]")
            target_matrix_C_ent_count = eval(f"X{target_matrix_index[m]}.shape[1]")
            sample[target_matrix_index[m]] = np.random.choice(target_matrix_R_ent_count * target_matrix_C_ent_count, int(0.2 * target_matrix_R_ent_count * target_matrix_C_ent_count) , replace = False) # sample 20 % of the data
    else:
        sample = np.array([]) # no sampling  
    #
    if target_matrix_index != -1:
        offset = {}
        for m in range(len(target_matrix_index)):
            offset[target_matrix_index[m]] = 0
            for i in range(target_matrix_index[m]):
                offset[target_matrix_index[m]] += eval(f"X{i}.shape[0] * X{i}.shape[1]")
    print(f"Offset for samples is {offset}")
    #
    if target_matrix_index != -1:
        for m in target_matrix_index:
            sample[m] = sample[m] + offset[m]
    else:
        sample = sample + offset 
    #
    if target_matrix_index != -1:
        test_data = pd.DataFrame()
        for m in sample.keys():
            test_data = pd.concat([test_data, full_df.iloc[sample[m]]], ignore_index=True)
    else:
        test_data = full_df.iloc[sample]   
    #
    orig_test_data_size = test_data.shape[0]
    #
    # Find all entries that are zeros in the test df
    zero_test_data = pd.DataFrame(test_data.groupby("left").sum()["value"]).reset_index()
    zero_test_data = zero_test_data[zero_test_data["value"] == 0]
    #
    test_data_np = test_data.to_numpy()
    #
    # Create link.dat.test
    number_of_pairs = 0
    for v in test_data.link_type.unique():
        link_file_for_lp = open(data_folder + f'sampled{sample_id}_link.dat.test.{v}', "w+")
        test_data_np = test_data[test_data["link_type"] == v].to_numpy()
        for i in range(test_data_np.shape[0]):
            number_of_pairs += 1
            link_file_for_lp.write(str(int(test_data_np[i][0])) + "\t" + str(int(test_data_np[i][1])) + "\t" + str(test_data_np[i][2]) + "\n")        
        link_file_for_lp.close()  
    #
    if target_matrix_index == -1:
        full_df.drop(sample, inplace = True, axis = 0)
    else:
        for m in sample.keys():
            full_df.drop(sample[m], inplace=True, axis=0)    
    #
    # Create link.dat file
    emb_df = full_df[full_df["value"] != 0]
    emb_df["value"] = emb_df["value"]
    emb_df["left"] = emb_df["left"].astype(int)
    emb_df["right"] = emb_df["right"].astype(int)
    emb_df["link_type"] = emb_df["link_type"].astype(int)
    emb_df = emb_df[["left", "right", "link_type", "value"]]
    emb_df.head()    
    #
    emb_df.to_csv(sampled_link_file, sep="\t", header=False, index=False)
    #
    if "Entity labels" in entity_df.columns:
        isNodeLabelSeparate = True
        print("Node labels are different from node types")
    else:
        isNodeLabelSeparate = False
        print("Node labels are the same as node types")
    #
    count = 0
    node_type = []
    node_label = []
    for entity in matrices_seen:
        print(f"Entity {entity}")
        entity_size = eval(f"{entity}_size")
        node_type_val = int(entity[1:])
        node_type.extend([node_type_val] * entity_size)
        if not isNodeLabelSeparate:
            node_label.extend([node_type_val] * entity_size)
        count += entity_size

    entity_df["Node label"] = node_label if not isNodeLabelSeparate else list(entity_df["Entity Labels"])
    entity_df["Node type"] = node_type 
    #
    test_labels_df = None
    if target_matrix_index != -1:
        test_labels_df = entity_df.sample(frac = 0.2, replace = False, random_state = rng)
        test_labels_df = test_labels_df[["entity_name", "Node type", "Node label"]]
        test_labels_df.head()
    #
    if target_matrix_index != -1:
        print(test_labels_df.shape)
    else:
        print("No test labels")
    #
    if target_matrix_index != -1:
        remaining_labels_df = entity_df.drop(test_labels_df.index)
        remaining_labels_df = remaining_labels_df[["entity_name", "Node type", "Node label"]]
        remaining_labels_df.head()
    else:
        remaining_labels_df = entity_df
    #
    if target_matrix_index != -1:
        test_labels_df.to_csv(sampled_label_test_file, sep = "\t", header = False)
    else:
        !> {sampled_label_test_file}  
    #
    remaining_labels_df.to_csv(sampled_label_file, sep = "\t", header = False)
    #
    entity_df[["entity_name", "Node type"]].to_csv(sampled_node_file, sep = "\t", header = False)
    #
    # Getting node counts
    node_values = list(entity_df.groupby("Node type").count()["entity_name"])
    print(node_values)
    #
    # Getting link counts
    emb_count = list(emb_df.groupby("link_type").count()["value"])
    if target_matrix_index != -1:
        for m in target_matrix_index:
            test_count = list(test_data[test_data["link_type"] == m].groupby("link_type").count()['value'])[0]
            emb_count[m] += test_count
    #print(emb_count)
    #
    # Getting label counts
    entity_df.groupby(["Node type", "Node label"]).count()
    label_counts = entity_df.groupby(['Node type','Node label']).count().apply(list).to_dict()
    #
    meta_file_writer = open(sampled_meta_file, "w+")

    meta_file_writer.write(f"Node Total: Count {sum(node_values)}" + "\n")
    for i in range(len(node_values)):
        meta_file_writer.write(f"Node Type_{i}: Count {node_values[i]}" + "\n")
    meta_file_writer.write(f"Edge Total: Count {sum(emb_count)}" + "\n")
    for i in range(len(emb_count)):
        meta_file_writer.write(f"Edge Type_{i}: Count {emb_count[i]}" + "\n")
    meta_file_writer.write(f"Label Total: Count {sum(label_counts['entity_name'].values())}" + "\n")

    for i in range(len(node_values)):
        meta_file_writer.write(f"Label Class_{i}_Total: Count {node_values[i]}" + "\n")
        for k in sorted(label_counts["entity_name"]):
            if k[0] == i:
                meta_file_writer.write(f"Label Class_{k[0]}_Type_{k[1]}: Count {label_counts['entity_name'][k]}" + "\n")

    meta_file_writer.close()
    #
    info_file_writer = open(sampled_info_file, "w+")

    info_file_writer.write("node.dat\n")
    info_file_writer.write("TYPE\tMEANING\n")
    for k, v in entity_name_mapping.items():
        info_file_writer.write(f"{k[1:]}"+"\t"+f"{v}"+"\n")

    info_file_writer.write("\n-----------------------------------------------\n")

    info_file_writer.write("link.dat\n")
    info_file_writer.write("LINK\tSTART\tEND\tMEANING\n")
    for k,v in entity_mapping.items():
        info_file_writer.write(f"{k[1:]}" + "\t" + f"{v[0][1:]}" + "\t" + f"{v[1][1:]}" + "\t" + f"{entity_name_mapping[f'E{int(v[0][1:])}']}" + "-and-" + f"{entity_name_mapping[f'E{int(v[1][1:])}']}" + "\n")

    info_file_writer.write("\n-----------------------------------------------\n")
    info_file_writer.write("label.dat\n")
    info_file_writer.write("TYPE\tCLASS\tMEANING\n")
    for i in range(len(node_values)):
        for k in sorted(label_counts["entity_name"]):
            if k[0] == i:
                info_file_writer.write(f"{i}" + "\t" + f"{k[1]}" + "\t" + f"{entity_name_mapping[f'E{int(k[1])}']}" + "\n")

    info_file_writer.close()   
    #
    pkl.dump(dict_id_idx, open(sampled_dict_id_idx_file,"wb"))

Shape of X0 (200, 500)
Shape of X1 (200, 300)
Shape of X2 (200, 400)
Shape of X3 (700, 500)
Shape of X4 (600, 300)
total_entities:  2700
E0_size:  200
E1_size:  500
E2_size:  300
E3_size:  400
E4_size:  700
E5_size:  600
Considering X0
E0 --> 0
E1 --> 200
Considering X1
E0 --> 0
E2 --> 700
Considering X2
E0 --> 0
E3 --> 1000
Considering X3
E4 --> 1400
E1 --> 200
Considering X4
E5 --> 2100
E2 --> 700
Offset for samples is {0: 0, 2: 160000}


/tmp/ipykernel_51698/3713062336.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emb_df["value"] = emb_df["value"]
/tmp/ipykernel_51698/3713062336.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emb_df["left"] = emb_df["left"].astype(int)
/tmp/ipykernel_51698/3713062336.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

Node labels are the same as node types
Entity E0
Entity E1
Entity E2
Entity E3
Entity E4
Entity E5
(540, 3)
[200, 500, 300, 400, 700, 600]
Shape of X0 (200, 500)
Shape of X1 (200, 300)
Shape of X2 (200, 400)
Shape of X3 (700, 500)
Shape of X4 (600, 300)
total_entities:  2700
E0_size:  200
E1_size:  500
E2_size:  300
E3_size:  400
E4_size:  700
E5_size:  600
Considering X0
E0 --> 0
E1 --> 200
Considering X1
E0 --> 0
E2 --> 700
Considering X2
E0 --> 0
E3 --> 1000
Considering X3
E4 --> 1400
E1 --> 200
Considering X4
E5 --> 2100
E2 --> 700
Offset for samples is {0: 0, 2: 160000}


/tmp/ipykernel_51698/3713062336.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emb_df["value"] = emb_df["value"]
/tmp/ipykernel_51698/3713062336.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emb_df["left"] = emb_df["left"].astype(int)
/tmp/ipykernel_51698/3713062336.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

Node labels are the same as node types
Entity E0
Entity E1
Entity E2
Entity E3
Entity E4
Entity E5
(540, 3)
[200, 500, 300, 400, 700, 600]
Shape of X0 (200, 500)
Shape of X1 (200, 300)
Shape of X2 (200, 400)
Shape of X3 (700, 500)
Shape of X4 (600, 300)
total_entities:  2700
E0_size:  200
E1_size:  500
E2_size:  300
E3_size:  400
E4_size:  700
E5_size:  600
Considering X0
E0 --> 0
E1 --> 200
Considering X1
E0 --> 0
E2 --> 700
Considering X2
E0 --> 0
E3 --> 1000
Considering X3
E4 --> 1400
E1 --> 200
Considering X4
E5 --> 2100
E2 --> 700
Offset for samples is {0: 0, 2: 160000}


...

Node labels are the same as node types
Entity E0
Entity E1
Entity E2
Entity E3
Entity E4
Entity E5
(540, 3)
[200, 500, 300, 400, 700, 600]
Shape of X0 (200, 500)
Shape of X1 (200, 300)
Shape of X2 (200, 400)
Shape of X3 (700, 500)
Shape of X4 (600, 300)
total_entities:  2700
E0_size:  200
E1_size:  500
E2_size:  300
E3_size:  400
E4_size:  700
E5_size:  600
Considering X0
E0 --> 0
E1 --> 200
Considering X1
E0 --> 0
E2 --> 700
Considering X2
E0 --> 0
E3 --> 1000
Considering X3
E4 --> 1400
E1 --> 200
Considering X4
E5 --> 2100
E2 --> 700
Offset for samples is {0: 0, 2: 160000}


/tmp/ipykernel_51698/3713062336.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emb_df["value"] = emb_df["value"]
/tmp/ipykernel_51698/3713062336.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emb_df["left"] = emb_df["left"].astype(int)
/tmp/ipykernel_51698/3713062336.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

Node labels are the same as node types
Entity E0
Entity E1
Entity E2
Entity E3
Entity E4
Entity E5
(540, 3)
[200, 500, 300, 400, 700, 600]
Shape of X0 (200, 500)
Shape of X1 (200, 300)
Shape of X2 (200, 400)
Shape of X3 (700, 500)
Shape of X4 (600, 300)
total_entities:  2700
E0_size:  200
E1_size:  500
E2_size:  300
E3_size:  400
E4_size:  700
E5_size:  600
Considering X0
E0 --> 0
E1 --> 200
Considering X1
E0 --> 0
E2 --> 700
Considering X2
E0 --> 0
E3 --> 1000
Considering X3
E4 --> 1400
E1 --> 200
Considering X4
E5 --> 2100
E2 --> 700
Offset for samples is {0: 0, 2: 160000}


/tmp/ipykernel_51698/3713062336.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emb_df["value"] = emb_df["value"]
/tmp/ipykernel_51698/3713062336.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emb_df["left"] = emb_df["left"].astype(int)
/tmp/ipykernel_51698/3713062336.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

Node labels are the same as node types
Entity E0
Entity E1
Entity E2
Entity E3
Entity E4
Entity E5
(540, 3)
[200, 500, 300, 400, 700, 600]


In [6]:
print("done")

done
